In [1]:
%run TEST/WZX/Test/send_email

StatementMeta(, , -1, Finished, Available)

请设置receiver_email、subject、body，以发送邮件


In [2]:
from datetime import datetime,timedelta    # import packages
import pyspark.sql.functions as F
import pandas as pd
import os

StatementMeta(aasparkpool001, 894, 3, Finished, Available)

In [3]:
mssparkutils.fs.mount( 
    "wasbs://bosch-dw-integration-layer@proddataplatcn3blob01.blob.core.chinacloudapi.cn", # connect blob
    "/mnt/bosch-dw-integration-layer",
    {"LinkedService" : "AzureBlobStorage_proddataplatcn3blob01"}
)

StatementMeta(aasparkpool001, 894, 4, Finished, Available)

True

In [4]:
receiver_email = ['external.Zhaoxiang.Wang@cn.bosch.com','zhaoxiang_wang@leansight.cn','fei.zhao2@cn.bosch.com'] # set email 
subject = 'Bosch Job Alarm: Travel Expense'
body = 'Travel Expense 机票数据未更新, 请及时确认————This is an alarm email from python. Please do not reply'

StatementMeta(aasparkpool001, 894, 5, Finished, Available)

In [7]:
print('==================travel expense: Air Ticket=====================')
def get_last_month():
    """ 
     * @ message : 获取上月与本月月份名
     * @ return   [list] 月份
    """
    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month
    previous_month = current_month - 1 if current_month > 1 else 12
    previous_year = current_year if current_month > 1 else current_year -1
    last_month = str(previous_year) + str(previous_month).zfill(2)
    now_month = str(current_year) + str(current_month).zfill(2)
    month_list = [last_month] # remove current month
    return month_list


def data_need(blobs):
    """ 
     * @ message : 获取需要的blob
     * @ param2   [type] blobs: blobs
     * @ return   [type]
    """
    need_blobs = []
    for i in blobs:
        split_blob = i.split('_')
        if len(split_blob) > 2:
            blob_name = i.split('_')[-2][:6]
        else:
            blob_name = i
        if blob_name not in get_last_month():
            continue
        need_blobs.append(i)
    return need_blobs

def hand_file():
    path = mssparkutils.fs.getMountPath("/mnt/bosch-dw-integration-layer")
    root = path + '/manual/AA_CTG/Air Tickets'
    files = os.listdir(root)
    files = data_need(files) # enble filter
    try:
        file_path = os.path.join(root,files[0])
        print(file_path)
        return file_path
    except:
        print('=======未识别到上月机票数据=========')
        send_to_email(receiver_email, subject, body)
        mssparkutils.notebook.exit()
file_path = hand_file()

StatementMeta(aasparkpool001, 894, 8, Finished, Available)

==================travel expense: Air Ticket=====================
/synfs/894/mnt/bosch-dw-integration-layer/manual/AA_CTG/Air Tickets/air_tickets_20231206_20240105.xlsx


In [9]:
ods_path = 'abfss://data-warehouse-ods@dlsaaddpnorth3001.dfs.core.chinacloudapi.cn/ods_mau_air_ticket_cn_mi.csv' # path
formatted_endtime = datetime.now()
formatted_endtime += timedelta(hours=8)
etl_load_time = formatted_endtime.strftime(f"%Y-%m-%d %H:%M:%S")    # etl_load_time

ods_df = spark.read.csv(ods_path,header=True).drop('no').drop('etl_load_time') # read
df = pd.read_excel(file_path,skiprows=[1],dtype=str)
df = spark.createDataFrame(df)

df1 = df.exceptAll(ods_df)  # handle
df2 = ods_df.unionAll(df1) \
         .withColumn('no',F.monotonically_increasing_id()) \
         .withColumn('etl_load_time',F.lit(etl_load_time)) \
         .toPandas() 

col_list = ods_df.columns # order by  
col_list.insert(0,'no')
df = df2[col_list]
display(df)

df.to_csv(ods_path,index=False, header=True) # save


StatementMeta(aasparkpool001, 894, 10, Finished, Available)

/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:604: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  [(c, t) for (_, c), t in zip(pdf_slice.iteritems(), arrow_types)]
/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:604: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  [(c, t) for (_, c), t in zip(pdf_slice.iteritems(), arrow_types)]


SynapseWidget(Synapse.DataFrame, e3c89178-6aef-4c31-8bd8-720ba2620d42)